Credit Risks
Step 1: Download the dataset from the following link:  https://www.openml.org/d/31

Step 2: Read the dataset into a pandas dataframe.

In [ ]:
import pandas as pd
from scipy.io import arff

data, meta = arff.loadarff("dataset_31_credit-g.arff")
df = pd.DataFrame(data)
print(df.head(6))


Step 3: Feature Selection
Choose the features relevant to our analysis.
Numeric Attributes: duration, age, residence_since, credit_amount
Nominal Attributes: credit_history, employment, job

In [ ]:
# Step 3: Feature Selection
numeric_features = ['duration', 'age', 'residence_since', 'credit_amount']
nominal_features = ['credit_history', 'employment', 'job']
selected_features = numeric_features + nominal_features

# Create a new DataFrame with selected features
df_selected = df[selected_features].copy()
print("Selected Features:")
print(df_selected.head())

Step 4: Preprocessing
Perform any needed pre-processing on the chosen features, including: Scaling, Encoding, Dealing with NaN values.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Encoding Nominal Features
# Using sparse_output=False for compatibility with pandas concatenation
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_nominal = encoder.fit_transform(df_selected[nominal_features])
# Create DataFrame for encoded variables with proper column names
encoded_nominal_df = pd.DataFrame(encoded_nominal, columns=encoder.get_feature_names_out(nominal_features))

# Scaling Numeric Features
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df_selected[numeric_features])
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric_features)

# Combine all features
df_processed = pd.concat([scaled_numeric_df, encoded_nominal_df], axis=1)

print("Processed Data Shape:", df_processed.shape)
print(df_processed.head())

Step 5: 